In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import string
import re
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix,accuracy_score, classification_report

data = pd.read_excel(r"C:\Users\elGhizi\Desktop\fakeNews\ProjetFinModule\backend\machineLearningModels\AJGT.xlsx")
print(data.head())
print(data.sample(5))

   ID                                               Feed Sentiment
0   1   اربد فيها جامعات اكثر من عمان ... وفيها قد عم...  Positive
1   2   الحلو انكم بتحكوا على اساس انو الاردن ما فيه ...  Negative
2   3                            كله رائع بجد ربنا يكرمك  Positive
3   4                                 لسانك قذر يا قمامه  Negative
4   5  ​انا داشره وغير متزوجه ولدي علاقات مشبوه واحشش...  Negative
        ID                                               Feed Sentiment
195    196  الدعوه مش بالكلام بس ممكن نكون دعاه باخلاقنا ا...  Positive
1523  1524  هذه مشكله الذي يغني كثيرا في الحمام ويتصور انه...  Negative
1754  1755            يارب يقوينا جميعا على طاعته وحسن عبادته  Positive
1785  1786  يعني هيك برامج بتبين للناس وين صرنا كم صارت نس...  Negative
207    208  الشعب الذي اختار نواب خرجين سجون سوابق بستاهل ...  Negative


In [3]:
punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ''' + string.punctuation

# Arabic stop words with nltk
stop_words = stopwords.words()

arabic_diacritics = re.compile("""
                             ّ    | # Shadda
                             َ    | # Fatha
                             ً    | # Tanwin Fath
                             ُ    | # Damma
                             ٌ    | # Tanwin Damm
                             ِ    | # Kasra
                             ٍ    | # Tanwin Kasr
                             ْ    | # Sukun
                             ـ     # Tatwil/Kashida
                         """, re.VERBOSE)

def preprocess(text):
    
    '''f
    text is an arabic string input
    
    the preprocessed text is returned
    '''
    
    #remove punctuations
    translator = str.maketrans('', '', punctuations)
    text = text.translate(translator)
    
    # remove Tashkeel
    text = re.sub(arabic_diacritics, '', text)
    
    #remove longation
    text = re.sub("[إأآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ؤ", "ء", text)
    text = re.sub("ئ", "ء", text)
    text = re.sub("ة", "ه", text)
    text = re.sub("گ", "ك", text)

    text = ' '.join(word for word in text.split() if word not in stop_words)

    return text
  
data['Feed'] = data['Feed'].apply(preprocess)
print(data.head(5))

   ID                                               Feed Sentiment
0   1  اربد جامعات اكثر عمان وفيها عمان ونص لعيبه الم...  Positive
1   2   الحلو انكم بتحكوا علي اساس انو الاردن فساد سرقات  Negative
2   3                            كله راءع بجد ربنا يكرمك  Positive
3   4                                    لسانك قذر قمامه  Negative
4   5  ​انا داشره وغير متزوجه ولدي علاقات مشبوه واحشش...  Negative


In [4]:
# splitting the data into target and feature
feature = data.Feed
target = data.Sentiment
# splitting into train and tests
X_train, X_test, Y_train, Y_test = train_test_split(feature, target, test_size =.2, random_state=100)
print(X_test)

1052                          فاشله بيءه فاشله
31               احيانا يكون الفشل دافع للنجاح
110                  اقل عادي مستواك اقلب وجهك
1216                       للمفاعل داخل الاردن
241                    الله اكبر احلاكم اشجعكم
                         ...                  
1718    غبي كوادر الاخوان المسلمين بقعه هالارض
1170                                  كلب واطي
1778                             يعطيك العافيه
1174          كلمه حب عجبتني جيت اقءلها جرحتني
1736     يارب اعيني علي دكرك وشكرك وحسن عبادتك
Name: Feed, Length: 360, dtype: object


In [5]:
pipe = make_pipeline(TfidfVectorizer(),
                    MultinomialNB())
pipe.fit(X_train,Y_train)
prediction = pipe.predict(X_test)
print(type(X_test))
print(f"Accuracy score is {accuracy_score(Y_test, prediction):.2f}")
print(classification_report(Y_test, prediction))

<class 'pandas.core.series.Series'>
Accuracy score is 0.84
              precision    recall  f1-score   support

    Negative       0.89      0.76      0.82       176
    Positive       0.80      0.91      0.85       184

    accuracy                           0.84       360
   macro avg       0.85      0.84      0.84       360
weighted avg       0.85      0.84      0.84       360



In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfvect = TfidfVectorizer(stop_words='arabic',max_df=0.7)
tfvect = TfidfVectorizer(max_df=0.8)
def fake_news_det(news):
    input_data = [news]
    vectorized_input_data = pd.Series(input_data)
    print(type(vectorized_input_data))
    prediction = pipe.predict(vectorized_input_data)
    print(prediction)

In [7]:
fake_news_det('يعطيك العافيه')

<class 'pandas.core.series.Series'>
['Positive']
